# Implementation of Boundary Equilibrium GANs
Reference: https://arxiv.org/pdf/1703.10717

In [ ]:
# Run the comment below only when using Google Colab
# !pip install torch torchvision

In [1]:
import torch
import torchvision
import torch.nn as nn

In [2]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
import numpy as np
import datetime
import os, sys

In [4]:
import glob

In [5]:
from PIL import Image

In [6]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imsave
%matplotlib inline

In [7]:
MODEL_NAME = 'BEGAN'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = "cpu"

In [8]:
IMAGE_SIZE = 248
IMAGE_DIM = (248, 248, 1)

In [9]:
def tensor2img(tensor):
    img = (np.transpose(tensor.detach().cpu().numpy(), [1,2,0])+1)/2.
    return img

In [10]:
def get_sample_image(G, n_noise):
    """
        save sample 25 images
    """
    z = torch.randn(25, n_noise).to(DEVICE)
    # print(z[0][0:100])
    y_hat = G(z).view(25, IMAGE_SIZE, IMAGE_SIZE)
    result = y_hat.cpu().data.numpy()
    img = np.zeros([IMAGE_SIZE * 5, IMAGE_SIZE * 5])
    for j in range(5):
        # b = np.concatenate([np.where(x > 0.5, 1, 0) for x in result[j*5:(j+1)*5]], axis=-1)
        b = np.concatenate([x for x in result[j*5:(j+1)*5]], axis=-1)
        img[j*IMAGE_SIZE:(j+1)*IMAGE_SIZE] = b
    return img

In [11]:
#def get_sample_image(G, n_noise=100, n_samples=64):
    """
        save sample 100 images
    """
    """
    n_rows = int(np.sqrt(n_samples))
    z = (torch.rand(size=[n_samples, n_noise])*2-1).to(DEVICE) # U[-1, 1]
    x_fake = G(z)
    x_fake = torch.cat([torch.cat([x_fake[n_rows*j+i] for i in range(n_rows)], dim=1) for j in range(n_rows)], dim=2)
    result = tensor2img(x_fake)
    return result
    """

'\n            n_rows = int(np.sqrt(n_samples))\n            z = (torch.rand(size=[n_samples, n_noise])*2-1).to(DEVICE) # U[-1, 1]\n            x_fake = G(z)\n            x_fake = torch.cat([torch.cat([x_fake[n_rows*j+i] for i in range(n_rows)], dim=1) for j in range(n_rows)], dim=2)\n            result = tensor2img(x_fake)\n            return result\n            '

In [12]:
class Encoder(nn.Module):
    """
        Convolutional Encoder
    """
    def __init__(self, in_channel=1, n_filters=128, hidden_dim=100):
        super(Encoder, self).__init__()
        assert IMAGE_DIM[0] % 2**2 == 0, 'Should be divided 4'
        self.flatten_dim = IMAGE_DIM[0]//2**2
        self.conv = nn.Sequential(
            # conv0
            nn.Conv2d(in_channel, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            # conv1
            nn.Conv2d(n_filters, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            # conv2
            nn.Conv2d(n_filters, n_filters*2, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_filters*2),
            nn.LeakyReLU(0.2),
            # conv3
            nn.Conv2d(n_filters*2, n_filters*2, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters*2),
            nn.LeakyReLU(0.2),
            # conv4
            nn.Conv2d(n_filters*2, n_filters*3, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_filters*3),
            nn.LeakyReLU(0.2),
            # conv5
            nn.Conv2d(n_filters*3, n_filters*3, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters*3),
            nn.LeakyReLU(0.2),
            # conv6
            nn.Conv2d(n_filters*3, n_filters*3, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters*3),
            nn.LeakyReLU(0.2),
#             nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(self.flatten_dim**2*n_filters*3, hidden_dim)
    
    def forward(self, x):
        h = self.conv(x)
        h = h.view(h.size(0), -1)
        h = self.fc(h)
        return h

In [13]:
class Decoder(nn.Module):
    """
        Convolutional Decoder
    """
    def __init__(self, out_channel=1, n_filters=128, n_noise=100):
        super(Decoder, self).__init__()
        assert IMAGE_DIM[0] % 2**2 == 0, 'Should be divided 4'
        self.flatten_dim = IMAGE_DIM[0]//2**2
        self.fc = nn.Sequential(
            nn.Linear(n_noise, self.flatten_dim**2*n_filters),
        )
        self.conv = nn.Sequential(
            # conv1
            nn.Conv2d(n_filters, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            # conv2
            nn.Conv2d(n_filters, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            nn.Upsample(scale_factor=2, mode='nearest'),
            # conv3
            nn.Conv2d(n_filters, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            # conv4
            nn.Conv2d(n_filters, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            nn.Upsample(scale_factor=2, mode='nearest'),
            # conv5
            nn.Conv2d(n_filters, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            # conv6
            nn.Conv2d(n_filters, n_filters, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(n_filters),
            nn.LeakyReLU(0.2),
            # conv6
            nn.Conv2d(n_filters, out_channel, 3, stride=1, padding=1, bias=True),
            nn.Tanh()
        )
    
    def forward(self, h):
        h = self.fc(h)
        h = h.view(h.size(0), -1, self.flatten_dim, self.flatten_dim)
        x = self.conv(h)
        return x

In [14]:
class Discriminator(nn.Module):
    """
        Convolutional Discriminator
    """
    def __init__(self, in_channel=1, n_filters=128, hidden_dim=64):
        super(Discriminator, self).__init__()
        self.encoder = Encoder(in_channel=in_channel, n_filters=n_filters, hidden_dim=hidden_dim)
        self.decoder = Decoder(out_channel=in_channel, n_filters=n_filters, n_noise=hidden_dim)
        
    def forward(self, x):
        h = self.encoder(x)
        x_ = self.decoder(h)
        return x_

In [15]:
class Generator(nn.Module):
    """
        Convolutional Generator
    """
    def __init__(self, out_channel=1, n_filters=128, n_noise=64):
        super(Generator, self).__init__()
        self.decoder = Decoder(out_channel=out_channel, n_filters=n_filters, n_noise=n_noise)
        
    def forward(self, h):
        x_ = self.decoder(h)
        return x_

In [16]:
transform = transforms.Compose([torchvision.transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                                # transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                transforms.Normalize(mean=(0.5),
                                #std=(0.5, 0.5, 0.5))
                                std=(0.5))
                               ]
)

CelebA-aligned download: [link](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)

In [17]:
# dataset = datasets.ImageFolder(root='geo-images-square', transform=transform)

In [18]:
batch_size = 5
n_noise = 248

In [19]:
# data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=8, pin_memory=True)

In [20]:
data_path = 'geo-images-square'
dataset = datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose([
            torchvision.transforms.Grayscale(num_output_channels=1),
            torchvision.transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])  
    )

In [21]:
data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )

In [22]:
D = Discriminator(in_channel=IMAGE_DIM[-1], hidden_dim=n_noise).to(DEVICE)
G = Generator(out_channel=IMAGE_DIM[-1], n_noise=n_noise).to(DEVICE)

In [23]:
D_opt = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
G_opt = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [24]:
# BEGAN causes mode collapse. it can be addressed by decaying lr
D_scheduler = torch.optim.lr_scheduler.MultiStepLR(D_opt, milestones=[3, 10, 17], gamma=0.6)
G_scheduler = torch.optim.lr_scheduler.MultiStepLR(G_opt, milestones=[3, 10, 17], gamma=0.6)

In [25]:
criterion = nn.L1Loss()

In [26]:
max_epoch = 20
step = 0

In [27]:
lr_k = 0.001
gamma = 0.7
k_t = 0
log_term = 100

In [28]:
steps_per_epoch = len(data_loader.dataset) // batch_size
steps_per_epoch

12

In [29]:
if not os.path.exists('samples'):
    os.makedirs('samples')
    
if not os.path.exists('ckpt'):
    os.makedirs('ckpt')

In [30]:
def save_checkpoint(state, file_name='checkpoint.pth.tar'):
    torch.save(state, file_name)

In [ ]:
m_lst = []
bestM = 1.
for epoch in range(max_epoch):
    for idx, (images, _) in enumerate(data_loader):
        G.zero_grad()
        # Training Discriminator
        x = images.to(DEVICE)
        x_outputs = D(x)
        D_x_loss = criterion(x_outputs, x)

        z = (torch.rand(size=[batch_size, n_noise])*2-1).to(DEVICE)
        x_fake = G(z)
        z_outputs = D(x_fake.detach())
        D_z_loss = criterion(z_outputs, x_fake)
        
        D_loss = D_x_loss - k_t*D_z_loss
        
        D.zero_grad()
        D_loss.backward()
        D_opt.step()

        # Training Generator
        z = (torch.rand(size=[batch_size, n_noise])*2-1).to(DEVICE)
        x_fake = G(z)
        z_outputs = D(x_fake)
        G_loss = criterion(x_fake, z_outputs)

        G.zero_grad()
        G_loss.backward()
        G_opt.step()
        
        bal = (gamma*D_x_loss - G_loss).detach()
        k_t = min(max(k_t + lr_k*bal, 0), 1)
        M_global = D_x_loss.detach() + torch.abs(bal)
        
        if M_global.item() < bestM:
            bestM = M_global.item()
            save_checkpoint({'global_step': step,
                 'D':D.state_dict(),
                 'G':G.state_dict(),
                 'd_optim': D_opt.state_dict(),
                 'g_optim' : G_opt.state_dict()},
                'ckpt/began{:06d}.pth.tar'.format(step))
        
        if step % log_term == 0:
            m_lst.append(M_global)
            dt = datetime.datetime.now().strftime('%H:%M:%S')
            print('Epoch: {}/{}, Step: {}, D Loss: {:.4f}, G Loss: {:.4f}, k: {:.4f}, M: {:.4f}, lr: {:.7f}, Time:{}'.format(epoch, max_epoch, step, D_loss.item(), G_loss.item(), k_t, M_global.item(), G_scheduler.get_lr()[0], dt))
            G.eval()
            img = get_sample_image(G, n_noise)
            imsave('samples/{}_step{:06d}.jpg'.format(MODEL_NAME, step), img, cmap='gray')
            G.train()
        
        step += 1
    D_scheduler.step()
    G_scheduler.step()

Epoch: 0/20, Step: 0, D Loss: 0.9089, G Loss: 0.3586, k: 0.0003, M: 1.1865, lr: 0.0002000, Time:17:35:24


In [ ]:
save_checkpoint({'global_step': step,
     'D':D.state_dict(),
     'G':G.state_dict(),
     'd_optim': D_opt.state_dict(),
     'g_optim' : G_opt.state_dict()},
    'ckpt/began{:06d}.pth.tar'.format(step))

### Random Sample

In [ ]:
G_path = sorted(glob.glob(os.path.join('ckpt', '*.pth.tar')))[-2]
state = torch.load(G_path)
G.load_state_dict(state['G'])

In [ ]:
G.eval()
None

In [ ]:
img = get_sample_image(G, n_noise, n_samples=25)
imshow(img)

In [ ]:
# Fake Image
idx = [3, 3]
row, col = IMAGE_DIM[0]*idx[0], IMAGE_DIM[1]*idx[1]
imshow(img[row:row+IMAGE_DIM[0], col:col+IMAGE_DIM[1], :])

In [ ]:
# Real Image
i = 14
rimg = tensor2img(images[i])
imshow(rimg)

### Interpolation

In [ ]:
def sample_noise(size=None):
    z = np.random.random(size=size)*2-1
    return z

In [ ]:
z_a, z_b = sample_noise(n_noise), sample_noise(n_noise)
zs = torch.tensor([np.linspace(z_a[i], z_b[i], num=10) for i in range(n_noise)], dtype=torch.float32).to(DEVICE)
zs = torch.transpose(zs, 0, 1)
zs.shape

In [ ]:
imgs = G(zs).detach()
imgs = torch.cat([imgs[i] for i in range(10)], dim=-1)
imgs.shape

In [ ]:
fig = plt.figure(figsize=(20, 3))
imshow(tensor2img(imgs))